# Recurrent neural network with numpy

## Prerequisite

In [1]:
# some important imports
import numpy as np
from tqdm import tqdm

## Encoding text

In [2]:
# data
text = open('data/toy.txt', 'r').read()

# text = 'Hallo'
text_length = len(text)
chars = list(set(text))
char_length = len(chars)
print('text is ', text_length, 'long and has ', char_length)

# creating training data
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

X = np.array([char_to_int[char] for char in text])
y = np.append(X[1:X.shape[0]], X[0])
print('first 10 datas: ', X[0:10])
print('first 10 labels: ', y[0:10])

text is  1993 long and has  56
first 10 datas:  [43 28 41 32  4 22 14  5 32  4]
first 10 labels:  [28 41 32  4 22 14  5 32  4 22]


## Forward pass


In [3]:
def forward_pass(X, hprev):
    ht, pt, loss = [hprev[0]], [], 0
    for t in range(len(X)):
        # creating a one hot encoded vector
        xt = np.zeros((char_length, 1))
        xt[X[t]] = 1

        # calculating forward pass
        zt = np.dot(Wxh, xt) + np.dot(Whh, ht[t])
        ht.append(np.tanh(zt))
        yt = np.dot(Why, ht[t])

        # getting probability distribution
        pt.append(np.exp(yt) / np.sum(np.exp(yt)))

        # summing up the loss of every output
        loss += -np.sum(np.log(pt[t][X[t]]))
    return ht, pt, loss / len(X)

## Backward pass

In [4]:
def backward_pass(X, y, ht, pt):
    dWhh, dWxh, dWhy = np.zeros_like(Whh), np.zeros_like(Wxh), np.zeros_like(Why)
    for t in reversed(range(len(X))):
        dout = pt.copy()
        dout[t][y[t]] -= 1
        dWhy += np.dot(dout[t], ht[t].T)
        dh = np.dot(Why.T, dout[t])
        dtanh = (1 - ht[t] * ht[t]) * dh
        xt = np.zeros((char_length, 1))
        xt[X[t]] = 1
        dWxh += np.dot(dtanh, xt.T)
        dWhh += np.dot(dtanh, ht[t - 1].T)
    
    dWhh /= len(X)
    dWxh /= len(X)
    dWhy /= len(X)
    # gradient clipping
    for dparam in [dWxh, dWhh, dWhy]:
        np.clip(dparam, -5, 5, out=dparam)
    return dWhh, dWxh, dWhy

## Predict function

In [5]:
def predict(X, Wxh, Whh, Why, hprev):
    ht, prediction = [hprev[0]], ''
    for t in range(len(X)):
        # creating a one hot encoded vector
        xt = np.zeros((char_length, 1))
        xt[X[t]] = 1

        # calculating forward pass
        zt = np.dot(Wxh, xt) + np.dot(Whh, ht[t])
        ht.append(np.tanh(zt))
        yt = np.dot(Why, ht[t])

        # getting probability distribution
        pt.append(np.exp(yt) / np.sum(np.exp(yt)))

        # creating a prediction string
        prediction += chars[np.argmax(pt[t])]
    return prediction

## Updating parameter with adagrad

### Intizializing hyperparameter

In [6]:
seq_size = 15
hidden_size = 200
learning_rate = 1e-1
epochs = 200

print('Training ', epochs, ' epochs with a sequence size of ', seq_size, ', a hidden size of ', hidden_size, ' and a learning rate of', learning_rate)

Training  200  epochs with a sequence size of  15 , a hidden size of  200  and a learning rate of 0.1


### Initizializing learnable parameter

In [7]:
Wxh = np.random.randn(hidden_size, char_length) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(char_length, hidden_size) * 0.01

In [8]:
# initializing hidden state and squared gradient
ht = [np.zeros((hidden_size, 1))]
grad_squared_xh, grad_squared_hh, grad_squared_hy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

loss = 0
for e in tqdm(range(epochs)):
    for steps in range(0, len(X), seq_size):
        inputs = X[steps:steps+seq_size]
        targets = y[steps:steps+seq_size]

        # forward and backward pass
        ht, pt, loss = forward_pass(inputs, ht)
        dWhh, dWxh, dWhy = backward_pass(inputs, inputs, ht, pt)
        
        # adagrad
        grad_squared_xh += dWxh ** 2
        grad_squared_hh += dWhh ** 2
        grad_squared_hy += dWhy ** 2
        
        # parameter update
        Wxh -= dWxh / np.sqrt(grad_squared_xh + 1e-7) * learning_rate
        Whh -= dWhh / np.sqrt(grad_squared_hh + 1e-7) * learning_rate
        Why -= dWhy / np.sqrt(grad_squared_hy + 1e-7) * learning_rate
    if e % 10 == 0:
        print('loss at epoch ', e, ' is ', loss)
        print(predict(X, Wxh, Whh, Why, ht))

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

loss at epoch  0  is  4.1273035967
gs iynon dndngII]kW
]
b
b
bâbÃ]â
.
.
b
bâbz]f
u
.
[âbjAW
u
.
b
bâb])Wpu
.
b
bâbâ]W]u
N
b
bâbâb(]â
.
.
b
bâ]ÃAu
.
.
[âbâbÃcu
.
.
bâbâbÃ
u
.
.
bâbj]W
u
.
b
bâbjAW
u
.
b
b
bjbÃ
u
.
.
bâbj]W
u
.
b
bâbjAâ
u
.
b
bâbj]W]u
.
.
bâbz]W
u
.
b
bâb
]W]u
.
.
b
bâbW]u
.
.
[
bâ]W]u
.
.
bâbj]W]u
.
.
b
bâbÃ]u
.
.
bâbâ]Ã]u
.
.
bâbj]W]u
p
b
bâb]AW]u
N
.
bâbzbÃAu
.
b
bâbI]W]L
)
.
b
bj]W
u
.
b
bâbjAâpu
.
b
b
bI]W
u
.
p
b
bj]W
u
.
p
bz]]]u
.
.
p
bâ]Ã]â
.
.
b
bj]W]u
u
.
[
bâ]Ã]â
.
.
b
bâ](]â
.
.
bâb
bâ]W
u
.
b
bâb(]â
u
.
b
bâbâ]W]w
u
.
b
bâbj)â
.
.
bâbâb]]f
u
.
bâb
b(Aâ
u
.
p
bâb(Aâ
u
.
b
b
bIAW
u
.
.
bâb]bW
u
.
b
bâbjAW
u
.
p
bâb(]â
.
.
b
bâbj]W]u
.
.
b
b(]W
u
.
.
bâb])â
u
.
b
bâb
]j]â
.
.
b
bâb]AW
u
.
b
bâb]]W]u
N
.
b
bjbW
u
.
b
bâbâb(]u
u
.
b
bâ]W]W
u
b
b
bâbâ]€)w
.
.
p
bz]W]u
.
.
b
bâbj]â
u
.
b
b
bj]â
.
.
b
bâbj)W
u
.
[
bâbW]â
u
.
b
bâb(]W]u
.
b
bâbâbW]u
.
.
I
bâb()â
.
.
b
bâ]]]u
.
.
b
bâb()â
u
.
b
bâ]W]u
.
.
b
bâb()â
.
.
b
bâb]]w
.
.
b
bâb]AW
u
.
b
b
bj]W
u
b
b
b
bâb(]w

  5%|████                                                                             | 10/200 [00:05<01:35,  1.99it/s]

loss at epoch  10  is  3.33348988261
gs,onhennnn ngIC”0I00j10"j”0lb9"0"j0j("jjb"b0zI©"j00jCTb"”€"€ÃI"j0jI"b(xj"j0jb"z0x€jj"b0I€"0bxb0x(jj"j"b"jIb”"I0jTI€"j""9b"x€"jI0"x9”x1b"b0jI0b€jb”00€"b90b0€j0b€jb"00€"b"jW"b9jb”00€"b00”zb0I0jb"jb""x""b90b0I9bWjT€bW"jIb€0b"€jIb0I00bb"j0b9j€”b"bzx(9b"jLIWjb0j€j0T0Ix"Ib0x0jj”""zjb"b0€Ixb0j€j0jb"jI0”0I(xIb0000b00jI""I""Ib0jb€"0"j0j("jjbb0jbI9x1b0Ix©jT0jb""j"j0jxb"b0jI0b€jb”00€"Ib0jb€b"x€"jIxb9jb”00€"b"jW""T0x0b"b9jI0b€jb”00€"b90b"TIb"x0I("bxb00bb”0x€"bhjW"€Ã00jI"b0I€"b000Ib0I0"IIb"bzx("jIb€jb”00€""b"b0jbI0Tb€j"”00€""b"b€"jL0z2b9j"”00€"1b"b€"T2x”0b9j"”00€"1b""jT0jb0jb000j"b€"j"b0"b0jW0jb0xb"I00j0jx"T000jb0zI©"j00jCTb020"jWl0"b"b"jI0b€jb”00€"b"jW0jI"b9€jb0I0"jxTj1b"j00jW"jjb"b"jIbz0xb""I""b0jb0jbj"0€"b"jW0€Exb€jb”00€"b9bb00xb0jjTx"xj0xb0x(IIx"b"j00j""jIb"jIbzjjIb0I9jxjT€€jb"jb0jI0I€"xb€bb"I0I”b00xb0"b0x00jb"jT€0I""jT0I02I0x1b0Ijb0x0j€jjIj”x"b0jb0zI©"j00jCb0"b0b(I"Ljb0"0Ib0jb00xb""L"9bb€bW00xb00z0b0x€"j"II0j0b0jb00xb0"bzx0"b9bb"”€j"Ã0jI"b€"jb"0(j02b"b00jj

 10%|████████                                                                         | 20/200 [00:09<01:26,  2.09it/s]

...

 90%|████████████████████████████████████████████████████████████████████████        | 180/200 [01:24<00:09,  2.14it/s]

loss at epoch  180  is  3.01800245691
gsim he ene ngIf  nhed nio nn ni  neenion  atinin ie eedd n nn ie  n oo  osnn  ho ne  aontr  n n nn     n  atn nehnion nntn  nn  o n   aninn   un tien  nu  n h un  hen  osenden  hen  honi nne en   n  ione nnne   nu n   ne nned ne nnun   n in nan nn n nnon n ienion n en en    io r tn   n n hn n sn nni  tueio irdi n  in n  neionnenion  tnn n nat   nehn n  nnee  nenn  eun  hen n n  nued hen  ue  hen  osedoirdm nann   te tien  nu oie n roio   ne  n   onniaon no   onn nntnne  n n  tndn nen  ue  hen  ani nn ni un  hen  anieiennie un  hen  aniunin  neun  hen  aoien too  nh  nahn   ni n  hn n  rneonnn  nenn  tinio ie eedheensenn  n nn  eun  hen  oseen mtuni  nenn i   r hennien  an maho  nn  i e an   ien  oseen   un  ion  nu hr  onu  n    end n n ao ienein tu m he   nnnn  aunnin  n  in   tnu  nin  nhdahn   ti ansanndan  enenn   ae   n   ien n  mtn   hnio ie e  n n nnhn annnnan aho  n nnn  nu io  n ie    h n n  een  hrd hn in i nu   n nnntn  uny   nnen  ahe 

 95%|████████████████████████████████████████████████████████████████████████████    | 190/200 [01:28<00:04,  2.14it/s]

loss at epoch  190  is  3.07315792354
gs o  o ene ngI,eethr  nhn n  n hnncnoinn t  inin hntne   n nnnhncte ti  csnn  ho nnehanntrn   naoo n  an  a n iein nn el    cn endna t lonn  auncohnn aon tn h uncohnn ansdn uncohnn ahoni nnis     n  hannann uc  nu n  ann nnn annnni ndsdn hn n n nn neoscn nchun n tn  e ne  d in c snnnan n hn nsnntenid aecir hn a n  hnancanehoncnsinn tlnn n n sndn  hn ns encn  l nn  auncohnn ean  nsndthnn  uncohnn ansddn ndnalann   uncohnn aon n  dn enin   oe asnd cunsaannnn st nnnnne ne  nnnd  r o eno  uncohnn eal nnnun aunc hnn eal  honoho uncohnn eal unin   auncohnn eaai   aan aehntra nsn n  n  in nn nn nncn  nnen   hnin hntne hn nsnnnn l nn  auncohnn ansd  n  unn lonl nh n dshnnn nn eal  ihod nn  han aoenshnn ansdsnnnaunc hnn aou ho  anuueo na asoi n  a shonnho  l  oin eannnsc  unn l an shl   anu dnho aes itn teei  ns nni nsdnn no    ensdn n hnnaeod  n ashnin hn n  n n nohlndnnnd n aho anconn  nu hod n ho t  iecnna e n ahod en i lionu  tn nnci    nn odnoinc  hec

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:33<00:00,  2.14it/s]
